# Dataset Explore

In [1]:
import pickle as pkl
import numpy as np
import os
from src.dataset import Multimodal_Datasets

import plotly.express as px
import plotly.subplots as sp
import plotly.figure_factory as ff
import pandas as pd

In [2]:
dataset_path = '/data1/multimodal/MulT/data/'

In [3]:
def barplot_label(data):

    label_list = []
    labels = data['labels']
    for idx, label in enumerate(labels):
        label = label[0][0]
        
        # min, max setting
        if label > 3.:
            label = 3
        elif label < -3.:
            label = -3

        # make class aligned list
        if label == 0.:
            label_list.append([label, 'neutral', round(label), data['id'][idx]])
        else:
            label_list.append([label, 'pos', round(label), data['id'][idx]]) if label > 0. else label_list.append([label, 'neg', round(label), data['id'][idx]])
    
    label_df = pd.DataFrame(label_list, columns=['values', 'binary', '7class', 'segment'])

    order = [-3, -2, -1, 0, 1, 2, 3]

    df_binary = label_df.groupby('binary').count().reset_index()
    df_7class = label_df.groupby('7class').count().reset_index()

    fig1 = px.bar(df_binary, x='binary', y='segment')
    fig2 = px.bar(df_7class, x='7class', y='segment')

    fig1_traces = []
    fig2_traces = []

    for trace in range(len(fig1["data"])):
        fig1_traces.append(fig1["data"][trace])
    for trace in range(len(fig2["data"])):
        fig2_traces.append(fig2["data"][trace])

    this_figure = sp.make_subplots(rows=1, cols=2, subplot_titles=("Binary", "7-class"))
    for traces in fig1_traces:
        this_figure.append_trace(traces, row=1, col=1)
    for traces in fig2_traces:
        this_figure.append_trace(traces, row=1, col=2)

    this_figure.update_xaxes(categoryorder='array', categoryarray= order)
    this_figure.update_layout(height=500, width=1200, title_text="Ground-truth distribution")
    # this_figure.update_yaxes(range=[0,420])
    this_figure.show()

## Unaligned set

In [4]:
# Unaligned
mosi_train_dataset = Multimodal_Datasets(dataset_path, data='mosi', split_type='train', if_align=False)
mosi_test_dataset = Multimodal_Datasets(dataset_path, data='mosi', split_type='test', if_align=False)
mosei_train_dataset = Multimodal_Datasets(dataset_path, data='mosei_senti', split_type='train', if_align=False)
mosei_test_dataset = Multimodal_Datasets(dataset_path, data='mosei_senti', split_type='test', if_align=False)

In [5]:
datasets = [mosi_train_dataset, mosi_test_dataset, mosei_train_dataset, mosei_test_dataset]

In [6]:
from tqdm import tqdm
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

@interact
def get_explore(idx=range(len(datasets))):
    name = datasets[idx].data + '_' + datasets[idx].split_type
    print("!! %s !!" % name.upper())
    print("number of modalities               :", datasets[idx].get_n_modalities())
    print("seq_len of (text, audio, vision)   :", datasets[idx].get_seq_len())
    print("get_dim of (text, audio, vision)   :", datasets[idx].get_dim())
    print("(number of labels, label_dim)      :", datasets[idx].get_lbl_info())

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3), value=0), Output()), _dom_classes=('wi…

In [7]:
with open(dataset_path+'mosi_data_noalign.pkl', 'rb') as f:
    mosi = pkl.load(f)
with open(dataset_path+'mosei_senti_data_noalign.pkl', 'rb') as f:
    mosei = pkl.load(f)

In [8]:
train_mosi = mosi['train']
train_mosei = mosei['train']
test_mosi = mosi['test']
test_mosei = mosei['test']

In [9]:
@interact
def get_mosi_sample(idx = range(len(test_mosi['text']))):
    print("TEXT", test_mosi['text'][idx].shape)
    print("VISION", test_mosi['vision'][idx].shape)
    print("AUDIO", test_mosi['audio'][idx].shape)
    print("LABEL", test_mosi['labels'][idx])
    print("ID", test_mosi['id'][idx])

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

In [10]:
@interact
def get_mosi_sample(idx = range(len(test_mosei['text']))):
    print("TEXT", test_mosei['text'][idx].shape)
    print("VISION", test_mosei['vision'][idx].shape)
    print("AUDIO", test_mosei['audio'][idx].shape)
    print("LABEL", test_mosei['labels'][idx])
    print("ID", test_mosei['id'][idx])

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

In [11]:
@interact
def get_mosi_sample(idx = range(len(train_mosei['text']))):
    print("TEXT", train_mosei['text'][idx].shape)
    print("VISION", train_mosei['vision'][idx].shape)
    print("AUDIO", train_mosei['audio'][idx].shape)
    print("LABEL", train_mosei['labels'][idx])
    print("ID", train_mosei['id'][idx])

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1…

In [15]:
barplot_label(test_mosei)

## Aligned set

In [22]:
# Aligned
mosi_train_aligned = Multimodal_Datasets(dataset_path, data='mosi', split_type='train', if_align=True)
mosi_test_aligned = Multimodal_Datasets(dataset_path, data='mosi', split_type='test', if_align=True)
mosei_train_aligned = Multimodal_Datasets(dataset_path, data='mosei_senti', split_type='train', if_align=True)
mosei_test_aligned = Multimodal_Datasets(dataset_path, data='mosei_senti', split_type='test', if_align=True)

In [23]:
aligned_datasets = [mosi_train_aligned, mosi_test_aligned, mosei_train_aligned, mosei_test_aligned]

In [25]:
@interact
def get_explore(idx=range(len(aligned_datasets))):
    name = aligned_datasets[idx].data + '_' + aligned_datasets[idx].split_type + '_aligned'
    print("!! %s !!" % name.upper())
    print("number of modalities               :", aligned_datasets[idx].get_n_modalities())
    print("seq_len of (text, audio, vision)   :", aligned_datasets[idx].get_seq_len())
    print("get_dim of (text, audio, vision)   :", aligned_datasets[idx].get_dim())
    print("(number of labels, label_dim)      :", aligned_datasets[idx].get_lbl_info())

interactive(children=(Dropdown(description='idx', options=(0, 1, 2, 3), value=0), Output()), _dom_classes=('wi…

In [ ]:
with open(dataset_path+'mosi_data.pkl', 'rb') as f:
    mosi = pkl.load(f)